## Download VGG from Gdrive and convert to tfrecord

In [ ]:
# !aws s3 cp s3://seamfix-machine-learning-nv02/DeepLearning/Mobilefacenet/512_embed.zip .
# !aws s3 cp data/ms1m_vgg.tfrecord s3://seamfix-machine-learning-nv02/DeepLearning/Mobilefacenet/Tfrecord/
# !aws s3 cp data/vgg.tfrecord s3://seamfix-machine-learning-nv02/DeepLearning/Mobilefacenet/Tfrecord/

In [10]:
# !pip install gdown
# !pip install mxnet
import os
import io
import cv2
import tqdm
import glob
import random
import numpy as np
# import mxnet as mx
from PIL import Image
from scipy import misc
import tensorflow as tf
from absl import logging

In [6]:
#mxrecords exploration
file_idx = "MS1M/faces_emore/train.idx"
file_rec = "MS1M/faces_emore/train.rec"
index = 0
id2range = {}
imgrec = mx.recordio.MXIndexedRecordIO(file_idx, file_rec, 'r')
item = imgrec.read_idx(index)
header, _ = mx.recordio.unpack(item)
print(header.label[0]) 
imgidx = list(range(1, int(header.label[0])))
seq_identity = range(int(header.label[0]), int(header.label[1]))
print(seq_identity[1] - seq_identity[0])
cnt = 0
for identity in seq_identity:
    cnt += 1
    s = imgrec.read_idx(identity)
    header, _ = mx.recordio.unpack(s)
    a, b = int(header.label[0]), int(header.label[1])
    id2range[identity] = (a, b)
print('id2range', len(id2range)) #no of labels eg 85742
print(len(imgidx)) # view no of data eg 5822653

KeyboardInterrupt: 

In [3]:
import tensorflow as tf

#reading TFrecords
def _parse_tfrecord(binary_img=False, is_ccrop=False):
    def parse_tfrecord(tfrecord):
        if binary_img:
            features = {'image/source_id': tf.io.FixedLenFeature([], tf.int64),
                        'image/data_source': tf.io.FixedLenFeature([], tf.string),
                        'image/encoded': tf.io.FixedLenFeature([], tf.string)}
            x = tf.io.parse_single_example(tfrecord, features)
            x_train = tf.image.decode_jpeg(x['image/encoded'], channels=3)
        else:
            features = {'image/source_id': tf.io.FixedLenFeature([], tf.int64),
                        'image/img_path': tf.io.FixedLenFeature([], tf.string)}
            x = tf.io.parse_single_example(tfrecord, features)
            image_encoded = tf.io.read_file(x['image/img_path'])
            x_train = tf.image.decode_jpeg(image_encoded, channels=3)

        y_train = tf.cast(x['image/source_id'], tf.float32)
        print(y_train)
#         x_train = _transform_images(is_ccrop=is_ccrop)(x_train)
        y_train = _transform_targets(y_train)
        return (x_train, y_train), y_train
    return parse_tfrecord

def _transform_images(is_ccrop=False):
    def transform_images(x_train):
        x_train = tf.image.resize(x_train, (128, 128))
        x_train = tf.image.random_crop(x_train, (112, 112, 3))
        x_train = tf.image.random_flip_left_right(x_train)
        x_train = tf.image.random_saturation(x_train, 0.6, 1.4)
        x_train = tf.image.random_brightness(x_train, 0.4)
        x_train = x_train / 255
        return x_train
    return transform_images


def _transform_targets(y_train):
    return y_train

def load_tfrecord_dataset(tfrecord_name, batch_size, binary_img=False, shuffle=True, buffer_size=10240,is_ccrop=False):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()
    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)
    dataset = raw_dataset.map(
        _parse_tfrecord(binary_img=binary_img, is_ccrop=is_ccrop),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(
        buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset


In [ ]:
tf.compat.v1.enable_eager_execution()
sum(1 for _ in tf.data.TFRecordDataset("data/final.tfrecord"))

In [4]:
raw_dataset = tf.data.TFRecordDataset("data/vgg.tfrecord")
raw_dataset = raw_dataset.repeat()
raw_dataset = raw_dataset.shuffle(buffer_size=10240)
dataset = raw_dataset.map(
    _parse_tfrecord(binary_img=True, is_ccrop=False),
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

Tensor("Cast:0", shape=(), dtype=float32)


In [23]:
image_features[1]

<tf.Tensor: shape=(), dtype=float32, numpy=77595.0>

## Version 1 method

##### Much slower and memory intensive

In [ ]:
#read data from mxnet into a sample list 
def mxrecordio_to_list(file_idx, file_rec, imgidx):
    imgrec = mx.recordio.MXIndexedRecordIO(file_idx, file_rec, 'r')
    samples = []
    for i in imgidx:
        item = imgrec.read_idx(i)
        header, s = mx.recordio.unpack(item)
        x = [header[1],s]
        samples.append(x)
        if (i % 100000) == 0:
            print(i)
    return samples

In [9]:
#Creating TFrecord
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def make_example(img_str, source_id):
    # Create a dictionary with features that may be relevant.
    feature = {'image/source_id': _int64_feature(source_id),
               'image/encoded': _bytes_feature(img_str)}

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
#gets the image and label from a list and returns a tfrecord
def main(_):
    samples = mxrecordio_to_list(file_idx, file_rec, imgidx)
    random.shuffle(samples)
    outputpath = "data/final.tfrecord"
    with tf.io.TFRecordWriter(outputpath) as writer:
        for id_name, img_path in tqdm.tqdm(samples):
            #key = img_path.split("seamfix-machine-learning/")[-1]
            tf_example = make_example(img_str=img_path, source_id=int(id_name))
            writer.write(tf_example.SerializeToString())

## Version 2 method (with shuffling)

##### Much faster and memory intensive 

In [10]:
#Creating TFrecord
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def make_example(img_str, source_id):
    # Create a dictionary with features that may be relevant.
    feature = {'image/source_id': _int64_feature(source_id),
               'image/encoded': _bytes_feature(img_str)}

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [14]:
#read data from mxnet into a sample list 
def mxrecordio_to_list(paths):
    total = 0
    samples = []
    for path in paths:
        file_idx = os.path.join(path, 'train.idx')
        file_rec = os.path.join(path, 'train.rec')
        imgrec = mx.recordio.MXIndexedRecordIO(file_idx, file_rec, 'r')
        
        s = imgrec.read_idx(0)
        header, _ = mx.recordio.unpack(s)
        imgidx = list(range(1, int(header.label[0])))
        max_len = len(imgidx)
        cnt = 0
        
        for i in imgidx:
            item = imgrec.read_idx(i)
            header_index, img = mx.recordio.unpack(item)
            x = [total + int(header_index.label), img]
            samples.append(x)
            cnt += 1
            print('%d/%d' % (cnt, max_len), end='\r')
            
        seq_identity = range(int(header.label[0]), int(header.label[1]))
        total = (len(seq_identity)) + total
    return samples

In [15]:
#gets the image and label from a list and returns a tfrecord
def main():
    print('write tfrecord from mxrec...')
    paths = ["faces_vgg_112/faces_vgg_112x112"] #["MS1M/faces_emore", "faces_vgg_112/faces_vgg_112x112"]
    samples = mxrecordio_to_list(paths)
    random.shuffle(samples)
    outputpath = "data/vgg.tfrecord"
    with tf.io.TFRecordWriter(outputpath) as writer:
        for id_name, img_path in tqdm.tqdm(samples):
            #key = img_path.split("seamfix-machine-learning/")[-1]
            tf_example = make_example(img_str=img_path, source_id=int(id_name))
            writer.write(tf_example.SerializeToString())

In [16]:
main()

write tfrecord from mxrec...


100%|██████████| 3137807/3137807 [07:18<00:00, 7148.17it/s] 


## Version 3 method (without shuffling)

##### Much faster and less memory intensive 

In [11]:
#read data from mxnet into a sample list 
def mxrecordio_to_list():
    total = 0
    samples = []
    outputpath = "data/ms1m_.tfrecord"
    paths = ["MS1M/faces_emore", "faces_vgg_112/faces_vgg_112x112"]
    with tf.io.TFRecordWriter(outputpath) as writer:
        for path in paths:
            file_idx = os.path.join(path, 'train.idx')
            file_rec = os.path.join(path, 'train.rec')
            imgrec = mx.recordio.MXIndexedRecordIO(file_idx, file_rec, 'r')

            s = imgrec.read_idx(0)
            header, _ = mx.recordio.unpack(s)
            imgidx = list(range(1, int(header.label[0])))
            max_len = len(imgidx)
            cnt = 0

            for i in tqdm.tqdm(imgidx):
                item = imgrec.read_idx(i)
                header_index, img = mx.recordio.unpack(item)
                tf_example = make_example(img_str=img, source_id=total + int(header_index.label))
                writer.write(tf_example.SerializeToString())
                cnt += 1
#                 print('%d/%d' % (cnt, max_len), end='\r')

            seq_identity = range(int(header.label[0]), int(header.label[1]))
            total = (len(seq_identity)) + total
    writer.close()

In [ ]:
mxrecordio_to_list()

 82%|████████▏ | 2580241/3137807 [58:05<11:18, 821.47it/s]  

## Version 4 method (with shuffling)

##### This peteryuX version. This works with images in directory rather than mxrecord format

In [8]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def make_example(img_str, source_id, filename):
    # Create a dictionary with features that may be relevant.
    feature = {'image/source_id': _int64_feature(source_id),
               'image/filename': _bytes_feature(filename),
               'image/encoded': _bytes_feature(img_str)}

    return tf.train.Example(features=tf.train.Features(feature=feature))


def main(dataset_path):

    if not os.path.isdir(dataset_path):
        logging.info('Please define valid dataset path.')
    else:
        logging.info('Loading {}'.format(dataset_path))

    samples = []
    logging.info('Reading data list...')
    for id_name in tqdm.tqdm(os.listdir(dataset_path)):
        img_paths = glob.glob(os.path.join(dataset_path, id_name, '*.jpg'))
        for img_path in img_paths:
            filename = os.path.join(id_name, os.path.basename(img_path))
            samples.append((img_path, id_name, filename))
    random.shuffle(samples)

    logging.info('Writing tfrecord file...')
    outputpath = "data/peteryuX_ms1m.tfrecord"
    with tf.io.TFRecordWriter(outputpath) as writer:
        for img_path, id_name, filename in tqdm.tqdm(samples):
            tf_example = make_example(img_str=open(img_path, 'rb').read(),
                                      source_id=int(id_name),
                                      filename=str.encode(filename))
            writer.write(tf_example.SerializeToString())

In [ ]:
main('data/MS1M/imgs')

 38%|███▊      | 2200520/5822653 [33:44<44:31, 1356.03it/s]  

In [2]:
import os 
len(os.listdir('data/MS1M/imgs'))

85742